In [ ]:
import pandas as pd


/Users/gustavetremey/Desktop/projet_python_films/projet_python_films/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:
##### Nouveau type de dataset: IMDB non commercial datasets, récupérationn de CSV opensource des films IMDB et des données de ratings ####
##Téléchargement des fichiers compressés TSV depuis IMDB: téléchargement en local à effectuer une fois car volumineux
title_basics_url= 'https://datasets.imdbws.com/title.basics.tsv.gz'
title_ratings_url= 'https://datasets.imdbws.com/title.ratings.tsv.gz'
title_crew_url='https://datasets.imdbws.com/title.crew.tsv.gz'
crew_names_url='https://datasets.imdbws.com/name.basics.tsv.gz'

df_imdb_title_id= pd.read_csv(title_basics_url, sep='\t', na_values='\\N') #données basiques sur films: thème, langues etc.
df_imdb_ratings = pd.read_csv(title_ratings_url, sep='\t', na_values='\\N') #notation des films
df_imdb_crew=pd.read_csv(title_crew_url, sep='\t', na_values='\\N') #réalisateurs et scénaristes
df_imdb_names=pd.read_csv(crew_names_url, sep='\t', na_values='\\N') #noms des réalisateurs et scénaristes en fonction de leur ID nconst


/var/folders/gk/2wm4vkt95dd9063d1r5_79pm0000gn/T/ipykernel_7408/2032448790.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imdb_title_id= pd.read_csv(title_basics_url, sep='\t', na_values='\\N') #données basiques sur films: thème, langues etc.


In [ ]:
#Import des dataframes des infos sur les réalisateurs et associations de leurs noms à leurs ID

df_imdb_names.rename(columns={'nconst':'directors'}, inplace=True) 

df_imdb_directors=pd.DataFrame()
df_imdb_directors=pd.merge(df_imdb_crew, df_imdb_names,on='directors', how='left') #Association nom ID

df_imdb_directors.drop(columns=['writers','knownForTitles'], inplace=True) #Cleaning des colonnes non pertinentes
df_imdb_directors.rename(columns={'directors':'directors_id','primaryName':'director_name'}, inplace=True) #distinction ID réalisateur

In [30]:
# Merge des dataframes titres et ratings pour associer aux films leurs notes IMDB avec titres et ID: plus de 1.6 million de films notés
df_imdb_ratings=df_imdb_ratings.copy()

df_imdb_ratings=pd.merge(df_imdb_ratings, df_imdb_title_id, on='tconst', how='left')

#Cleaning du dataframe merged
df_imdb_ratings.drop(columns=['primaryTitle'], inplace=True)
df_imdb_ratings.sort_values(by='startYear')





,tconst,averageRating,numVotes,titleType,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
1161119,tt3155794,6.8,2457,short,Passage de Venus,0,1874.0,NaN,1.0,"Documentary,History,Short"
736210,tt14495706,5.7,477,short,La Rosace Magique,0,1878.0,NaN,1.0,"Animation,Short"
1174022,tt32173095,5.2,83,short,Les Clowns,0,1878.0,NaN,1,"Animation,Short"
1174019,tt32173050,4.8,89,short,L'Amazone,0,1878.0,NaN,1,"Animation,Short"
1174016,tt32173003,5.1,95,short,Le Jeu de Grâces,0,1878.0,NaN,1,"Animation,Short"
...,...,...,...,...,...,...,...,...,...,...
1621859,tt9593410,6.9,15,short,Aufbruch: Corporhate Murder,0,NaN,NaN,3.0,"Music,Short"
1623038,tt9623504,6.8,12,short,The Legendary Kid Combo: I wanna know,0,NaN,NaN,3.0,"Music,Short"
1626431,tt9710724,8.5,17,movie,Bay Lawz: Stick to the Code,0,NaN,NaN,110.0,Drama
1630327,tt9823122,5.6,22,tvSeries,Mannphodganj Ki Binny,0,NaN,NaN,NaN,"Comedy,Drama,Romance"


In [ ]:
#Réduction du champ d'études du dataframe aux longs métrages
mask_movies=df_imdb_ratings["titleType"]=="movie"
df_movie_ratings=df_imdb_ratings.loc[mask_movies]
df_movie_ratings.drop(columns=['titleType','endYear'], inplace=True)
df_movie_ratings.rename(columns={'startYear':'release_year'}, inplace=True)

#Association des réalisateurs aux films

df_movie_ratings_complete=df_movie_ratings.copy()
df_movie_ratings_complete=pd.merge(df_movie_ratings_complete, df_imdb_directors, on='tconst',how='left')
df_movie_ratings_complete.reset_index(drop=True, inplace=True)
df_movie_ratings_complete.head()

/var/folders/gk/2wm4vkt95dd9063d1r5_79pm0000gn/T/ipykernel_7408/1834143833.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie_ratings.drop(columns=['titleType','endYear'], inplace=True)
/var/folders/gk/2wm4vkt95dd9063d1r5_79pm0000gn/T/ipykernel_7408/1834143833.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie_ratings.rename(columns={'startYear':'release_year'}, inplace=True)


In [ ]:
################ Prémices data analysis: obtenir les 100 films les mieux notés par genre #####################
##############################################################################################################

def get_top_n_movies_per_genre(df, n=100, rating_col='averageRating', genre_col='genres'):
    """
    Returns:
        Un DataFrame contenant les N films les mieux notés pour chaque genre.
    """
    # Utiliser groupby sur la colonne de genre et appliquer nlargest sur la colonne de note
    # Le paramètre 'keep' par défaut dans nlargest est 'first', ce qui est généralement suffisant.
    df=df.loc[df['numVotes']>=10000]

    top_n_movies = df.groupby(genre_col).apply(
        lambda x: x.nlargest(n, rating_col)
    ).reset_index(drop=True)

    return top_n_movies

df_top_100_movies_per_genre = get_top_n_movies_per_genre(df_movie_ratings_complete, n=100)


/var/folders/gk/2wm4vkt95dd9063d1r5_79pm0000gn/T/ipykernel_7408/599078295.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_n_movies = df.groupby(genre_col).apply(
